# Wildfires in Brazil

The Amazon forest has been hit hard by wildfires in recent years. This project entails the analysis of historical data concerning wildfires in Brazil and the causes behind them.

The present book contains the cleaning and treatment of the acquired data for further analysis and visualization.

## Data cleaning

The data was obtained from the following sources through csv files or web scraping:

1. Wildfires data for each state in Brazil per month in the period 1998-2017. Source: PORTAL BRASILEIRO DE DADOS ABERTOS. http://www.dados.gov.br/dataset/sistema-nacional-de-informacoes-florestais-snif 
2. Deforestation of the Brazilian Amazon Forest in the period 2004-2019. Source: Coordenação-Geral de Observação da Terra. http://www.obt.inpe.br/OBT/assuntos/programas/amazonia/prodes 
3. Precipitation (rain level) in Brazil in the period 1998-2017. Data source: Kaggle, from Banco de Dados Meteorológicos para Ensino e Pesquisa https://www.kaggle.com/fabiopotsch/precipitation-in-brazil
4. Temperature in Brazil in the period 1991-2016. Source: World Bank Data https://climateknowledgeportal.worldbank.org/download-data

The following data was manually copied given the low number of entries needed and can be found in the "clean_data" folder with the title "5_production.csv":

- Palm oil and soy production in the period 1998-2019. Source: Index Mundi. https://www.indexmundi.com/agriculture/?country=br&commodity=soybean-meal&graph=production and https://www.indexmundi.com/agriculture/?country=br&commodity=palm-oil&graph=production  
- Forest area loss in the period 1998-2014. Data source: Wold Bank Data https://data.worldbank.org/indicator/AG.LND.FRST.ZS?view=chart

All the aqcquired data was loaded into Tableau and analyzed initially. The joint data was then saved as a csv file and can be found in the "clean_data" folder with the title "joint.csv". Finally, the Brazilian states were classified as Amazon and non-Amazon states and added to the data frame as a new column (entry 6 of this notebook). The final data frame can be found in the "clean_data" folder with the title "all_data.csv"

In [1]:
# Import the necessary libraries

import pandas as pd
import numpy as np
import lxml.html as lh
import requests

### 1. Wildfires data for each state in Brazil per month in the period 1998-2017.

In [2]:
# Import the data 

fires_year_state = pd.read_csv("../Data/1_rf_incendiosflorestais_focoscalor_estados_1998-2017.csv")

In [3]:
# Check for missing values

fires_year_state.isnull().sum()

year      0
state     0
month     0
number    0
date      0
dtype: int64

In [4]:
# Convert "number" column to integer

fires_year_state["number"] = fires_year_state.number.astype('int64')

In [5]:
# Replace the column "month" with month id's
# First check the spelling of the months in Portuguese

fires_year_state["month"].unique()

array(['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho',
       'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro'],
      dtype=object)

In [6]:
# Create a dictionary with the months id's

months = {"Janeiro": 1, 
          "Fevereiro": 2, 
          "Março": 3, 
          "Abril": 4, 
          "Maio": 5, 
          "Junho": 6, 
          "Julho": 7, 
          "Agosto": 8, 
          "Setembro": 9, 
          "Outubro": 10, 
          "Novembro": 11, 
          "Dezembro": 12}

In [7]:
# Replace the column "month" with month id's

fires_year_state = fires_year_state.replace({"month": months})

In [8]:
# Create a new column with country name

fires_year_state["country"] = "Brazil"

In [9]:
fires_year_state

,year,state,month,number,date,country
0,1998,Acre,1,0,01-01-98,Brazil
1,1999,Acre,1,0,01-01-99,Brazil
2,2000,Acre,1,0,01-01-00,Brazil
3,2001,Acre,1,0,01-01-01,Brazil
4,2002,Acre,1,0,01-01-02,Brazil
...,...,...,...,...,...,...
6448,2012,Tocantins,12,128,01-01-12,Brazil
6449,2013,Tocantins,12,85,01-01-13,Brazil
6450,2014,Tocantins,12,223,01-01-14,Brazil
6451,2015,Tocantins,12,373,01-01-15,Brazil


In [10]:
# Verify the data for each state

pd.Series(fires_year_state["state"]).value_counts()

Alagoas                239
Rio de Janeiro         239
Mato Grosso            239
Bahia                  239
Paraná                 239
Roraima                239
Sao Paulo              239
Rondonia               239
Rio Grande do Norte    239
Piauí                  239
Distrito Federal       239
Pará                   239
Maranhao               239
Rio Grande do Sul      239
Paraiba                239
Ceara                  239
Minas Gerais           239
Goias                  239
Santa Catarina         239
Pernambuco             239
Amapa                  239
Tocantins              239
Acre                   239
Mato Grosso do Sul     239
Amazonas               239
Espirito Santo         239
Sergipe                239
Name: state, dtype: int64

In [11]:
# We can see that Brazil's 27 states are present, and all states have the same entries

In [12]:
fires_year_state

,year,state,month,number,date,country
0,1998,Acre,1,0,01-01-98,Brazil
1,1999,Acre,1,0,01-01-99,Brazil
2,2000,Acre,1,0,01-01-00,Brazil
3,2001,Acre,1,0,01-01-01,Brazil
4,2002,Acre,1,0,01-01-02,Brazil
...,...,...,...,...,...,...
6448,2012,Tocantins,12,128,01-01-12,Brazil
6449,2013,Tocantins,12,85,01-01-13,Brazil
6450,2014,Tocantins,12,223,01-01-14,Brazil
6451,2015,Tocantins,12,373,01-01-15,Brazil


In [ ]:
# Save the data frame to a csv file

fires_year_state.to_csv("../Data/clean_data/1_fires.csv", index=False)

### 2. Deforestation data for Brazil's Legal Amazon (BLA) per year in the period 2004-2019.

Instructions from:
https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [13]:
# Import the data

url = "http://www.obt.inpe.br/OBT/assuntos/programas/amazonia/prodes"

In [14]:
#Create a handle, page, to handle the contents of the website

page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [15]:
tr_elements = doc.xpath('//tr') #Create empty list
col = []
i = 0 #For each row, store each first element (header) and an empty list
#strting from tr_elements[2] to have the correct headers
for t in tr_elements[2]:
    i += 1
    name = t.text_content()
    print ('%d: %s' % (i,name))
    col.append((name,[]))

1: Ano/Estados
2: AC
3: AM
4: AP
5: MA
6: MT
7: PA
8: RO
9: RR
10: TO
11: AMZ LEGAL


In [16]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(3,len(tr_elements)):
    #T is our j'th row
    T = tr_elements[j]
    
    #If row is not of size 11, the //tr data is not from our table 
    if len(T) != 11:
        break
    
    #i is the index of our column
    i = 0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data = t.text_content() 
        #Check if row is empty
        if i > 0:
        #Convert any numerical value to integers
            try:
                data = int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i += 1

In [17]:
dicti = {title:column for (title,column) in col}

In [18]:
dicti = {title:column for (title,column) in col}
df = pd.DataFrame(dicti)

In [19]:
df

,Ano/Estados,AC,AM,AP,MA,MT,PA,RO,RR,TO,AMZ LEGAL
0,2004,728,1232,46,755,11814,8870,3858,311,158,27772
1,2005,592,775,33,922,7145,5899,3244,133,271,19014
2,2006,398,788,30,674,4333,5659,2049,231,124,14286
3,2007,184,610,39,631,2678,5526,1611,309,63,11651
4,2008,254,604,100,1271,3258,5607,1136,574,107,12911
5,2009,167,405,70,828,1049,4281,482,121,61,7464
6,2010,259,595,53,712,871,3770,435,256,49,7000
7,2011,280,502,66,396,1120,3008,865,141,40,6418
8,2012,305,523,27,269,757,1741,773,124,52,4571
9,2013,221,583,23,403,1139,2346,932,170,74,5891


In [ ]:
# Save to csv file

df.to_csv("../Data/2_deforestation_yearly_amazon.csv ", index=False)

In [20]:
# A new csv file was saved in the "clean_data" folder as "2_deforestation.csv" 
# after transposing the data frame to obtain the states as entry values 

### 3. Precipitation data for Brazil's Legal Amazon (BLA) per year in the period 2004-2019.

In [21]:
# Import the data 

precip = pd.read_csv("../Data/3_precipitation_brazil.csv")

In [22]:
# Check for missing values

precip.isnull().sum()

state            0
date             0
precipitation    0
dtype: int64

In [23]:
precip.columns

Index(['state', 'date', 'precipitation'], dtype='object')

In [24]:
precip["state"].unique()

array(['BA', 'RR', 'SE', 'AL', 'TO', 'GO', 'PI', 'MG', 'PR', 'MA', 'AP',
       'RJ', 'AC', 'AM', 'DF', 'PE', 'CE', 'PA', 'MT', 'PB', 'RS', 'RN',
       'SP', 'ES', 'SC'], dtype=object)

In [25]:
# Create a dictionary with the states id's

states = {"BA": "Bahia", 
          "RR": "Roraima", 
          "SE": "Sergipe", 
          "AL": "Alagoas", 
          "TO": "Tocantins", 
          "GO": "Goias", 
          "PI": "Piauí", 
          "MG": "Minas Gerais", 
          "PR": "Paraná", 
          "MA": "Maranhao", 
          "AP": "Amapa", 
          "RJ": "Rio de Janeiro",
          "AC": "Acre", 
          "AM": "Amazonas", 
          "DF": "Distrito Federal", 
          "PE": "Pernambuco", 
          "CE": "Ceara", 
          "PA": "Pará", 
          "MT": "Mato Grosso", 
          "PB": "Paraiba", 
          "RS": "Rio Grande do Sul", 
          "RN": "Rio Grande do Norte", 
          "SP": "Sao Paulo", 
          "ES": "Espirito Santo", 
          "SC": "Santa Catarina"}

In [26]:
# Replace the column "state" with state names

precip = precip.replace({"state": states})

In [27]:
precip["date"] = precip.date.astype('datetime64[ns]')

In [28]:
precip.dtypes

state                    object
date             datetime64[ns]
precipitation           float64
dtype: object

In [29]:
# Get the month and year in separate columns

precip['year'] = pd.DatetimeIndex(precip['date']).year
precip['month'] = pd.DatetimeIndex(precip['date']).month

In [ ]:
# Save the data frame to a csv file

precip.to_csv("../Data/clean_data/3_precipitation.csv", index=False)

### 4. Temperature in Brazil per month from 1991 to 2016.

In [30]:
# Load the data

temp = pd.read_csv("../Data/4_temperature_1991_2016_BRA.csv")

In [31]:
temp.head()

,Temperature - (Celsius),Year,Statistics,Country,ISO3
0,25.6309,1991,Jan Average,Brazil,BRA
1,25.9331,1991,Feb Average,Brazil,BRA
2,25.6195,1991,Mar Average,Brazil,BRA
3,25.3122,1991,Apr Average,Brazil,BRA
4,24.6685,1991,May Average,Brazil,BRA


In [32]:
# Check for missing values

temp.isnull().sum()

Temperature - (Celsius)    0
 Year                      0
 Statistics                0
 Country                   0
 ISO3                      0
dtype: int64

In [33]:
# Checking datatypes

temp.dtypes

Temperature - (Celsius)    float64
 Year                        int64
 Statistics                 object
 Country                    object
 ISO3                       object
dtype: object

In [34]:
temp['month'] = temp[' Statistics'].str.replace(" Average","")
temp

,Temperature - (Celsius),Year,Statistics,Country,ISO3,month
0,25.6309,1991,Jan Average,Brazil,BRA,Jan
1,25.9331,1991,Feb Average,Brazil,BRA,Feb
2,25.6195,1991,Mar Average,Brazil,BRA,Mar
3,25.3122,1991,Apr Average,Brazil,BRA,Apr
4,24.6685,1991,May Average,Brazil,BRA,May
...,...,...,...,...,...,...
307,25.5629,2016,Aug Average,Brazil,BRA,Aug
308,25.9775,2016,Sep Average,Brazil,BRA,Sep
309,27.0781,2016,Oct Average,Brazil,BRA,Oct
310,26.7037,2016,Nov Average,Brazil,BRA,Nov


In [35]:
# Create column month 

temp = temp.rename(columns = {"Temperature - (Celsius)" : "temperature", 
                              " Year" : "year",
                              " Country" : "country"}).drop(columns = [" Statistics", " ISO3"])


In [36]:
temp

,temperature,year,country,month
0,25.6309,1991,Brazil,Jan
1,25.9331,1991,Brazil,Feb
2,25.6195,1991,Brazil,Mar
3,25.3122,1991,Brazil,Apr
4,24.6685,1991,Brazil,May
...,...,...,...,...
307,25.5629,2016,Brazil,Aug
308,25.9775,2016,Brazil,Sep
309,27.0781,2016,Brazil,Oct
310,26.7037,2016,Brazil,Nov


In [37]:
months = {" Jan": 1, 
          " Feb": 2, 
          " Mar": 3, 
          " Apr": 4, 
          " May": 5, 
          " Jun": 6, 
          " Jul": 7, 
          " Aug": 8, 
          " Sep": 9, 
          " Oct": 10, 
          " Nov": 11, 
          " Dec": 12}

In [38]:
# Replace the column "month" with month id's

temp = temp.replace({"month": months})

In [39]:
temp.dtypes

temperature    float64
year             int64
country         object
month            int64
dtype: object

In [40]:
temp

,temperature,year,country,month
0,25.6309,1991,Brazil,1
1,25.9331,1991,Brazil,2
2,25.6195,1991,Brazil,3
3,25.3122,1991,Brazil,4
4,24.6685,1991,Brazil,5
...,...,...,...,...
307,25.5629,2016,Brazil,8
308,25.9775,2016,Brazil,9
309,27.0781,2016,Brazil,10
310,26.7037,2016,Brazil,11


In [ ]:
# Save the data frame to a csv file

temp.to_csv("../Data/clean_data/4_temperature.csv", index=False)

### 5. Production of palm oil and soy in Brazil in the period 1998-2019

In [41]:
# The data was manually copied into an excel file 
# given the low number of entries needed 
# the file can be found in the "clean_data" folder with the title "5_production.csv"

### 6. Classification of Amazon or non-Amazon states

In [42]:
fires_year_state = pd.read_csv("../Data/clean_data/joint.csv")

In [43]:
fires_year_state

,Country,state,Year,Month,fires,forest_area,defo_area,co2,temperature,precipitation,palm,soy
0,Brazil,Acre,1998,1,0,62.975756,NaN,312289.054,26.7041,335.5,93,16651
1,Brazil,Acre,1999,1,0,62.671491,NaN,320173.104,26.0825,463.6,108,16478
2,Brazil,Acre,2000,1,0,62.367225,NaN,327983.814,25.8922,188.8,110,17725
3,Brazil,Acre,2001,1,0,62.019301,NaN,337433.673,25.8890,317.2,118,19407
4,Brazil,Acre,2002,1,0,61.671377,NaN,332266.870,26.0038,217.3,129,21449
...,...,...,...,...,...,...,...,...,...,...,...,...
6448,Brazil,Tocantins,2012,12,128,59.401972,52.0,470028.726,26.5734,227.9,340,27310
6449,Brazil,Tocantins,2013,12,85,59.284243,74.0,503677.118,26.3653,433.0,370,28540
6450,Brazil,Tocantins,2014,12,223,59.166513,50.0,529808.160,26.4223,247.2,400,31300
6451,Brazil,Tocantins,2015,12,373,59.048784,57.0,NaN,27.0235,149.4,415,30750


In [44]:
fires_year_state_new = fires_year_state.assign(Amazon_noneAmazon = 'not_amazon' )

In [45]:
fires_year_state_new

,Country,state,Year,Month,fires,forest_area,defo_area,co2,temperature,precipitation,palm,soy,Amazon_noneAmazon
0,Brazil,Acre,1998,1,0,62.975756,NaN,312289.054,26.7041,335.5,93,16651,not_amazon
1,Brazil,Acre,1999,1,0,62.671491,NaN,320173.104,26.0825,463.6,108,16478,not_amazon
2,Brazil,Acre,2000,1,0,62.367225,NaN,327983.814,25.8922,188.8,110,17725,not_amazon
3,Brazil,Acre,2001,1,0,62.019301,NaN,337433.673,25.8890,317.2,118,19407,not_amazon
4,Brazil,Acre,2002,1,0,61.671377,NaN,332266.870,26.0038,217.3,129,21449,not_amazon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6448,Brazil,Tocantins,2012,12,128,59.401972,52.0,470028.726,26.5734,227.9,340,27310,not_amazon
6449,Brazil,Tocantins,2013,12,85,59.284243,74.0,503677.118,26.3653,433.0,370,28540,not_amazon
6450,Brazil,Tocantins,2014,12,223,59.166513,50.0,529808.160,26.4223,247.2,400,31300,not_amazon
6451,Brazil,Tocantins,2015,12,373,59.048784,57.0,NaN,27.0235,149.4,415,30750,not_amazon


In [46]:
# first, set all Amazon_noneAmazon string to "not_amazon"
# then
# if the state name is one of the following, change the Amazon_none_Amazon column to "Amazon"


fires_year_state_new.loc[fires_year_state_new['state'].str.contains('Acre'), 'Amazon_noneAmazon'] = 'Amazon'
fires_year_state_new.loc[fires_year_state_new['state'].str.contains('Amapa'), 'Amazon_noneAmazon'] = 'Amazon'
fires_year_state_new.loc[fires_year_state_new['state'].str.contains('Amazonas'), 'Amazon_noneAmazon'] = 'Amazon'
fires_year_state_new.loc[fires_year_state_new['state'].str.contains('Pará'), 'Amazon_noneAmazon'] = 'Amazon'
fires_year_state_new.loc[fires_year_state_new['state'].str.contains('Rondonia'), 'Amazon_noneAmazon'] = 'Amazon'
fires_year_state_new.loc[fires_year_state_new['state'].str.contains('Roraima'), 'Amazon_noneAmazon'] = 'Amazon'
fires_year_state_new.loc[fires_year_state_new['state'].str.contains('Tocantis'), 'Amazon_noneAmazon'] = 'Amazon'
fires_year_state_new.loc[fires_year_state_new['state'].str.contains('Mato Grosso'), 'Amazon_noneAmazon'] = 'Amazon'
fires_year_state_new.loc[fires_year_state_new['state'].str.contains('Maranhao'), 'Amazon_noneAmazon'] = 'Amazon'

In [47]:
fires_year_state_new

,Country,state,Year,Month,fires,forest_area,defo_area,co2,temperature,precipitation,palm,soy,Amazon_noneAmazon
0,Brazil,Acre,1998,1,0,62.975756,NaN,312289.054,26.7041,335.5,93,16651,Amazon
1,Brazil,Acre,1999,1,0,62.671491,NaN,320173.104,26.0825,463.6,108,16478,Amazon
2,Brazil,Acre,2000,1,0,62.367225,NaN,327983.814,25.8922,188.8,110,17725,Amazon
3,Brazil,Acre,2001,1,0,62.019301,NaN,337433.673,25.8890,317.2,118,19407,Amazon
4,Brazil,Acre,2002,1,0,61.671377,NaN,332266.870,26.0038,217.3,129,21449,Amazon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6448,Brazil,Tocantins,2012,12,128,59.401972,52.0,470028.726,26.5734,227.9,340,27310,not_amazon
6449,Brazil,Tocantins,2013,12,85,59.284243,74.0,503677.118,26.3653,433.0,370,28540,not_amazon
6450,Brazil,Tocantins,2014,12,223,59.166513,50.0,529808.160,26.4223,247.2,400,31300,not_amazon
6451,Brazil,Tocantins,2015,12,373,59.048784,57.0,NaN,27.0235,149.4,415,30750,not_amazon


In [48]:
# Checking the data

fires_year_state_new.loc[fires_year_state_new['state'].str.contains('Mato Grosso')]

,Country,state,Year,Month,fires,forest_area,defo_area,co2,temperature,precipitation,palm,soy,Amazon_noneAmazon
2390,Brazil,Mato Grosso,1998,1,0,62.975756,NaN,312289.054,26.7041,217.1,93,16651,Amazon
2391,Brazil,Mato Grosso,1999,1,39,62.671491,NaN,320173.104,26.0825,281.6,108,16478,Amazon
2392,Brazil,Mato Grosso,2000,1,44,62.367225,NaN,327983.814,25.8922,173.7,110,17725,Amazon
2393,Brazil,Mato Grosso,2001,1,44,62.019301,NaN,337433.673,25.8890,119.0,118,19407,Amazon
2394,Brazil,Mato Grosso,2002,1,172,61.671377,NaN,332266.870,26.0038,217.6,129,21449,Amazon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2863,Brazil,Mato Grosso do Sul,2012,12,194,59.401972,NaN,470028.726,26.5734,NaN,340,27310,Amazon
2864,Brazil,Mato Grosso do Sul,2013,12,195,59.284243,NaN,503677.118,26.3653,NaN,370,28540,Amazon
2865,Brazil,Mato Grosso do Sul,2014,12,84,59.166513,NaN,529808.160,26.4223,NaN,400,31300,Amazon
2866,Brazil,Mato Grosso do Sul,2015,12,394,59.048784,NaN,NaN,27.0235,NaN,415,30750,Amazon


In [ ]:
# Save data frame with new column

fires_year_state_new.to_csv("../Data/clean_data/all_data.csv ", index=False)